# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
file_path = "../WeatherPy/output_data/citiesdata.csv"
cities_df = pd.read_csv(file_path)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lázaro Cárdenas,17.9583,-102.2000,24.61,75,82,1.70,MX,1619496156
1,Avila,40.5833,-5.0000,8.33,89,56,0.89,ES,1619496171
2,Bredasdorp,-34.5322,20.0403,14.00,88,97,1.54,ZA,1619496039
3,Ushuaia,-54.8000,-68.3000,6.00,87,40,1.03,AR,1619495977
4,Jaisinghnagar,23.7000,81.3833,35.71,6,76,1.75,IN,1619496172


In [15]:
cities_df.shape

(563, 9)

In [16]:
cities_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,5.630000e+02
mean,20.435275,15.153830,15.283321,70.220249,53.760213,3.201901,1.619496e+09
std,33.599577,90.533356,11.023214,22.444581,37.807920,2.137327,9.363382e+01
min,-54.800000,-179.166700,-13.890000,1.000000,0.000000,0.210000,1.619496e+09
25%,-8.076950,-66.552950,6.110000,60.000000,17.000000,1.540000,1.619496e+09
50%,25.021300,21.548200,16.670000,75.000000,57.000000,2.600000,1.619496e+09
75%,48.898550,92.825000,24.435000,87.000000,90.000000,4.120000,1.619496e+09
max,78.218600,179.316700,37.060000,100.000000,100.000000,12.860000,1.619496e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [18]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Store humudity in rating
rating = cities_df["Humidity"].astype(float)

In [19]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
max_temp = 26.66
min_temp = 21.11
max_wind_speed = 10
ideal_cloudiness = 0

ideal_cities_df = cities_df.loc[(cities_df['Max Temp'] < max_temp) & (cities_df['Max Temp'] > min_temp) & (cities_df['Wind Speed'] < max_wind_speed) & (cities_df['Cloudiness'] == ideal_cloudiness)]

ideal_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,Buraidah,26.3260,43.9750,24.00,53,0,2.06,SA,1619496200
219,Yangi Marg`ilon,40.4272,71.7189,23.00,33,0,1.03,UZ,1619496240
248,Riyadh,24.6877,46.7219,25.00,22,0,3.88,SA,1619495955
275,Yulara,-25.2406,130.9889,25.00,22,0,5.66,AU,1619496172
491,Umluj,25.0213,37.2685,24.64,66,0,2.06,SA,1619496334
539,Koungou,-12.7336,45.2042,25.00,94,0,0.51,YT,1619496350
551,Moree,-29.4667,149.8500,25.00,31,0,0.51,AU,1619496353


In [21]:
ideal_cities_df.shape

(7, 9)

In [22]:
ideal_cities_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,7.000000,7.000000,7.000000,7.000000,7.0,7.000000,7.000000e+00
mean,7.003043,75.103914,24.520000,45.857143,0.0,2.244286,1.619496e+09
std,28.539988,46.227025,0.765593,26.767073,0.0,1.910522,1.416150e+02
min,-29.466700,37.268500,23.000000,22.000000,0.0,0.510000,1.619496e+09
25%,-18.987100,44.589600,24.320000,26.500000,0.0,0.770000,1.619496e+09
50%,24.687700,46.721900,25.000000,33.000000,0.0,2.060000,1.619496e+09
75%,25.673650,101.353900,25.000000,59.500000,0.0,2.970000,1.619496e+09
max,40.427200,149.850000,25.000000,94.000000,0.0,5.660000,1.619496e+09


In [23]:
ideal_cities_df = ideal_cities_df.dropna()

ideal_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,Buraidah,26.3260,43.9750,24.00,53,0,2.06,SA,1619496200
219,Yangi Marg`ilon,40.4272,71.7189,23.00,33,0,1.03,UZ,1619496240
248,Riyadh,24.6877,46.7219,25.00,22,0,3.88,SA,1619495955
275,Yulara,-25.2406,130.9889,25.00,22,0,5.66,AU,1619496172
491,Umluj,25.0213,37.2685,24.64,66,0,2.06,SA,1619496334
539,Koungou,-12.7336,45.2042,25.00,94,0,0.51,YT,1619496350
551,Moree,-29.4667,149.8500,25.00,31,0,0.51,AU,1619496353


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# set up a data frame to hold summary information
columns_names = ["Hotel Name", "Lat", "Lng", "City", "Country"]
hotel_df = pd.DataFrame(columns = columns_names)

# geocoordinates
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_cities_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    country = row["Country"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat}, {lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request
    response = requests.get(base_url, params=params)
    
    # convert to json
    response_json = response.json()
    
    # data frame
    hotel_df = hotel_df.append({"Hotel Name":response_json['results'][0]['name'], 
                                    "Lat":lat, 
                                    "Lng":lng,
                                    "City":city,
                                    "Country":country}, ignore_index = True)

In [25]:
print(json.dumps(response_json, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJl97dqRITtAN334yDb5rAJNRsraa2x0KJfeT7UX2rxaCysflinNfulAbS1WxbQ9vZ9-9vghGKgkVhaGD8ZtWIKLembQbOsqI68TTuuIs8y586XPisrtsZfPhVrUja-ZJqz9wNE59ueWIP8ShTiaSsvLKNmWunznnRdMppf9mFBth6xwaGNT1QTUaL4UvTqcVJBD3xGZPj5vE0U6X6HZudPeJy9xjZXwyKXslXgq3sUh12SOL0oAWlXA2xwQFhIQbMVxYNZY4-Z-axS0451zC4BccJa3Axl6a-qpsMCyOB7DlL8u5I58Te7zSFh4S-3quKm7JKc4wDXOBZBgKZjidVf7gYVp6Fp5LDi7xOvVcRxukjJpkynG6MS3TM1IxVQ6RGBjh-H-EKEOa3jPGYxD7pq9vNkyXzhg2BHuEMTn4KEYf-7bxV8WpSy",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -29.4727198,
                    "lng": 149.8462736
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.4713716197085,
                        "lng": 149.8474282802915
                    },
                    "southwest": {
                        "lat": -29.47406958029149

In [26]:
hotel_df.head()

,Hotel Name,Lat,Lng,City,Country
0,فندق راحة السلام,26.3260,43.9750,Buraidah,SA
1,Ikathouse,40.4272,71.7189,Yangi Marg`ilon,UZ
2,"InterContinental Riyadh, an IHG Hotel",24.6877,46.7219,Riyadh,SA
3,Desert Gardens Hotel - Ayers Rock Resort,-25.2406,130.9889,Yulara,AU
4,Moon light Furnished Units,25.0213,37.2685,Umluj,SA


In [27]:
hotel_df.shape

(7, 5)

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))